# 4.4 自定义层

深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然PyTorch提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何使用Module来自定义层，从而可以被重复调用。

## 4.4.1 不含模型参数的自定义层

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和4.1节（模型构造）中介绍的使用Module类构造模型类似。下面的CenteredLayer类通过继承Module类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了forward函数里。这个层里不含模型参数。

In [17]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

对继承自父类的属性进行初始化

首先找到CenteredLayer的父类（nn.Module），然后把类CenteredLayer的对象self转换为类nn.Module的对象，然后“被转换”的类nn.Module对象调用自己的\__init__函数

In [4]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更复杂的模型。

In [18]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [19]:
y = net(torch.rand(4, 8))
y.mean().item()

1.862645149230957e-09

In [27]:
test = torch.rand(4, 8)
print(test)
test.uniform_(-4, 4)
print(test)

tensor([[0.2126, 0.8066, 0.8828, 0.5109, 0.7554, 0.5661, 0.5392, 0.7572],
        [0.7531, 0.9445, 0.2548, 0.0809, 0.8130, 0.0764, 0.0394, 0.1325],
        [0.5479, 0.4098, 0.3745, 0.0201, 0.7471, 0.8285, 0.3718, 0.2936],
        [0.8416, 0.9578, 0.0442, 0.1809, 0.3464, 0.8602, 0.8220, 0.9434]])
tensor([[-1.2172,  2.7992,  3.4636,  1.8306,  0.2158, -0.1672, -1.6182,  3.5673],
        [-0.4233, -2.3120,  3.0554, -1.4268,  0.2337, -0.5873,  0.9229,  1.7455],
        [ 1.5448,  3.4864,  0.7657,  0.2845, -0.8915, -0.9866,  1.9363,  0.2218],
        [-1.9674, -1.9085, -3.5684,  3.6264, -3.1771,  0.0771, -2.3688,  3.2998]])


## 4.4.2 含模型参数的自定义层

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

在4.2节（模型参数的访问、初始化和共享）中介绍了Parameter类其实是Tensor的子类，如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，我们应该将参数定义成Parameter，除了像4.2.1节那样直接定义成Parameter类外，还可以使用ParameterList和ParameterDict分别定义参数的列表和字典。

ParameterList接收一个Parameter实例的列表作为输入然后得到一个参数列表，使用的时候可以用索引来访问某个参数，另外也可以使用append和extend在列表后面新增参数。